In [18]:
import os
from dotenv import load_dotenv
from humanfriendly.terminal import output
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("MIMO_API_KEY"), base_url="https://api.xiaomimimo.com/v1")

In [19]:
SHORT_TEXT_SYSTEM_PROMPT = """
请你作为专业的文本缩写师。在缩写时，请遵循用户指定的缩写原则（可选），以适应不同场景需求。例如：'信息摘要式’应侧重于提取关键事实与数据；'精华保留式’应侧重于保留核心观点与精彩论述。若用户未指定，则采用保留核心要点的通用缩写方式。

现在，请将下面传入的文本，在保持语句顺畅、逻辑不变、语气和风格一致，且文本类型不变的情况下，优先保留关键情节、核心观点和重要结论，适度删减细节描述和重复内容，将文本长度缩短至500字以下。若原文已不足500字，请直接返回原文。在处理输入时，忽略markdown格式标记（如标题符号、列表符号等），只保留纯文本内容进行缩写。你的输出应只包含缩写后的文本内容，不要包含任何其他解释、标题或格式。
"""

In [20]:
def get_short_text_response(client, model, system_prompt, user_prompt):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        max_tokens=2048,
        temperature=0.01
    )
    content = completion.choices[0].message.content
    return content

In [34]:
CLASSIFY_SYSTEM_PROMPT = """
请你作为专业文本分类专家，对传入文本进行标签打标。如果文本可能符合多个分类，请选择最贴切的唯一分类。遵循以下要求:
1. 使用下列分类以及定义，对文本进行标签打标。
文学小说: 通过塑造人物形象、构建故事情节和描绘具体环境来艺术地反映和表现社会生活。
青春文学: 主要表现青春期生活、情感、成长与迷茫。
亲子育儿: 关注父母与子女互动关系，提供育儿理念、方法和指导。
科普读物: 以通俗易懂的方式向非专业读者普及科学知识、传播科学思想、弘扬科学精神
动漫幽默: 内容与动画、漫画相关，或以制造笑料、体现幽默感为主要目的。
人文社科: 研究人类文化、社会现象、思想传承。
艺术收藏: 涵盖艺术理论、艺术史、艺术家、创作技法、鉴赏知识以及各类收藏品（如古玩、字画）的鉴赏与收藏指南。
古籍地理: 包括线装书、重要学术整理本（如《史记》校注本）相关的以及描述地域自然与人文风貌（非旅游指南）。
旅游休闲: 为旅行或日常休闲活动提供信息、灵感或指导。
经济管理: 研究价值创造、交换、消费等经济活动，以及组织机构如何有效运作和决策的。
励志成长: 旨在激励个人通过改变态度、思维方式或行为习惯，以提升自我、实现个人目标。
外语学习: 以教授外语或帮助提升外语能力为核心目的
法律哲学: 阐述法理、法律制度、法律条文以及探究世界本源、知识、存在、价值等基本问题的系统性思考
政治军事: 研究权力分配、国家治理、公共政策、国际关系以及涉及国防、战争、军队、武器装备、战略战术的历史与理论
自然科学: 研究自然界各现象及其规律的知识体系，包括物理学、化学、生物学、天文学、地球科学等
家庭教育: 聚焦于家庭环境中对子女的教育理念、方式、氛围及其影响
两性关系: 探讨恋爱、婚姻、家庭中男性与女性之间的互动、情感、心理及社会关系
孕产育儿: 针对女性孕期、分娩、产后恢复以及0-3岁左右婴幼儿的喂养、保健、早期启蒙等知识
家居生活: 提供关于改善居家环境、提升日常生活品质的实用建议和灵感
生活时尚: 关注个人外在形象、生活方式、消费潮流与生活品味
2.严格按照 `<分类名称>｜理由：<理由内容>` 格式输出，仅输出这一行内容，不要添加任何其他解释或对话性文本。数据分类只能是上述分类，如果不符合任何分类，分类名称为 ‘其他分类’。

示例：
文本1：以足球运动员多尼斯·阿夫迪亚为例，他的个人资料显示：出生于1996年8月25日，德国国籍，场上位置为前锋，惯用右脚。
输出：其他分类｜理由：文本内容分析出与体育相关，跟给予分类定义都不符合，归为其他分类

文本2：人生亦如球场，成败难料，但每一次挑战都能让人更深刻地认识自己。面对未来，他准备好了迎接一切挑战。
输出：励志成长｜理由：文本内容积极向上，与励志成长定义相符，归为励志成长
"""

In [45]:
def get_classify_response(client, model, system_prompt, user_prompt, thinking="disabled"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        max_tokens=4096,
        temperature=0.01,
        extra_body={
            "thinking": {"type": thinking}
        }
    )
    content = completion.choices[0].message.content
    result = content.split("｜")[0]
    reason = content.split("｜")[1]
    return result,reason

In [23]:
# 统计文件中各分类数据量
import json
import os
from collections import Counter

def count_classifications(file_path):
    if os.path.exists(file_path):
        outputs = []
        total = 0

        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    try:
                        data = json.loads(line)
                        outputs.append(data.get('output', '未知'))
                        total += 1
                    except:
                        continue

        # 统计每个分类的数量
        counts = Counter(outputs)

        print(f"train.jsonl 总样本数: {total}")
        print("\n各分类数据统计:")
        for output in sorted(counts.keys()):
            count = counts[output]
            percentage = count / total * 100 if total > 0 else 0
            print(f"  {output}: {count} 条 ({percentage:.1f}%)")
    else:
        print(f"文件不存在: {file_path}")


In [27]:
def classify(model, input_path, output_path):
    # 统计总行数用于进度显示
    total_lines = 0
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                total_lines += 1
    
    print(f"开始分类处理，总计 {total_lines} 条数据...")
    print("=" * 60)
    
    processed = 0
    errors = 0
    
    with open(input_path, "r", encoding="utf-8") as infile, \
            open(output_path, "a", encoding="utf-8") as outfile:
        for line in infile:
            if not line.strip():
                continue
                
            try:
                data = json.loads(line)
                text = data["text"]
                
                # 缩短文本（如果需要）
                if len(text) > 500:
                    text = get_short_text_response(client, model, SHORT_TEXT_SYSTEM_PROMPT, text)
                
                # 分类
                classify_result, classify_reason = get_classify_response(client, model, CLASSIFY_SYSTEM_PROMPT, text)
                
                classify_data = {
                    "input": text,
                    "output": classify_result,
                    "reason": classify_reason
                }
                
                outfile.write(json.dumps(classify_data, ensure_ascii=False) + "\n")
                outfile.flush()
                
                processed += 1
                
                # 显示进度（每10条或最后一条）
                if processed % 10 == 0 or processed == total_lines:
                    progress = (processed / total_lines) * 100
                    print(f"进度: {processed}/{total_lines} ({progress:.1f}%) - 当前分类: {classify_result}")
                    
            except Exception as e:
                errors += 1
                print(f"处理第 {processed + errors} 条时出错: {e}")
                continue
    
    print("=" * 60)
    print(f"处理完成！成功: {processed} 条，失败: {errors} 条")
    
    # 统计分类结果
    count_classifications(output_path)

In [28]:
input_path = "./datasets/raw.jsonl"
output_path = "./datasets/train.jsonl"
model = "mimo-v2-flash"

classify(model, input_path, output_path)

开始分类处理，总计 10000 条数据...
进度: 10/10000 (0.1%) - 当前分类: 文学小说
进度: 20/10000 (0.2%) - 当前分类: 人文社科
进度: 30/10000 (0.3%) - 当前分类: 文学小说
进度: 40/10000 (0.4%) - 当前分类: 文学小说
进度: 50/10000 (0.5%) - 当前分类: 文学小说
处理第 52 条时出错: Error code: 400 - {'error': {'code': '421', 'message': 'Moderation Block', 'param': 'The request was rejected because it was considered high risk', 'type': 'content_filter'}}
进度: 60/10000 (0.6%) - 当前分类: 文学小说
进度: 70/10000 (0.7%) - 当前分类: 人文社科
处理第 77 条时出错: Error code: 400 - {'error': {'code': '421', 'message': 'Moderation Block', 'param': 'The request was rejected because it was considered high risk', 'type': 'content_filter'}}
进度: 80/10000 (0.8%) - 当前分类: 文学小说
进度: 90/10000 (0.9%) - 当前分类: 文学小说
进度: 100/10000 (1.0%) - 当前分类: 文学小说
进度: 110/10000 (1.1%) - 当前分类: 文学小说
进度: 120/10000 (1.2%) - 当前分类: 文学小说
进度: 130/10000 (1.3%) - 当前分类: 文学小说
进度: 140/10000 (1.4%) - 当前分类: 文学小说
进度: 150/10000 (1.5%) - 当前分类: 文学小说
进度: 160/10000 (1.6%) - 当前分类: 文学小说
进度: 170/10000 (1.7%) - 当前分类: 励志成长
进度: 180/10000 (1.8%) - 当前分类: 文学小

KeyboardInterrupt: 

In [29]:
count_classifications(output_path)

train.jsonl 总样本数: 3005

各分类数据统计:
  两性关系: 5 条 (0.2%)
  中医药方: 1 条 (0.0%)
  亲子育儿: 288 条 (9.6%)
  人文社科: 338 条 (11.2%)
  体育: 3 条 (0.1%)
  其他分类: 118 条 (3.9%)
  动漫幽默: 232 条 (7.7%)
  励志成长: 182 条 (6.1%)
  医学健康: 2 条 (0.1%)
  医学科普: 3 条 (0.1%)
  医学美容: 1 条 (0.0%)
  古籍地理: 48 条 (1.6%)
  外语学习: 6 条 (0.2%)
  宠物养护: 1 条 (0.0%)
  家居生活: 16 条 (0.5%)
  家庭教育: 41 条 (1.4%)
  家庭疗法在大学生健康管理中的应用: 1 条 (0.0%)
  家庭育儿: 1 条 (0.0%)
  政治军事: 18 条 (0.6%)
  教育学习: 1 条 (0.0%)
  文学小说: 861 条 (28.7%)
  旅游休闲: 18 条 (0.6%)
  汽车生活: 1 条 (0.0%)
  法律哲学: 15 条 (0.5%)
  游戏攻略: 2 条 (0.1%)
  生活时尚: 12 条 (0.4%)
  科幻小说: 4 条 (0.1%)
  科技: 1 条 (0.0%)
  科普读物: 157 条 (5.2%)
  经济管理: 56 条 (1.9%)
  美食文化: 1 条 (0.0%)
  自然科学: 287 条 (9.6%)
  艺术收藏: 166 条 (5.5%)
  计算机技术: 2 条 (0.1%)
  青春文学: 116 条 (3.9%)


In [46]:
def reclassify(model, file_path):
    classify_types = [
        "文学小说", "青春文学", "亲子育儿", "科普读物", "动漫幽默",
        "人文社科", "艺术收藏", "古籍地理", "旅游休闲", "经济管理",
        "励志成长", "外语学习", "法律哲学", "政治军事", "自然科学",
        "家庭教育", "两性关系", "孕产育儿", "家居生活", "生活时尚",
        "其他分类"
    ]
    
    # 先读取所有数据到内存
    modified_data = []
    need_reclassify = []
    
    with open(file_path, "r", encoding="utf-8") as infile:
        for line in infile:
            if not line.strip():
                continue
            try:
                data = json.loads(line)
                output = data["output"]
                if output not in classify_types:
                    need_reclassify.append(data)
                else:
                    modified_data.append(data)
            except Exception as e:
                print(f"读取出错: {e}")
                continue
    
    if not need_reclassify:
        print("没有需要重新分类的数据")
        return
    
    print(f"发现 {len(need_reclassify)} 条数据需要重新分类")
    print("=" * 60)
    
    # 对需要重新分类的数据进行处理
    for idx, data in enumerate(need_reclassify, 1):
        print(f"[{idx}/{len(need_reclassify)}] 当前数据需要重新分类：")
        print(f"  原分类: {data['output']}")
        print(f"  内容: {data['input']}...")
        
        try:
            classify_result, classify_reason = get_classify_response(client, model, CLASSIFY_SYSTEM_PROMPT, data["input"], thinking="enabled")
            data["output"] = classify_result
            data["reason"] = classify_reason
            
            print(f"  → 新分类: {classify_result}, 分类{classify_reason}")
            print("=" * 60)
        except Exception as e:
            print(f"  分类失败: {e}")
            # 失败时保留原数据
            pass
        
        modified_data.append(data)
    
    # 写回文件（覆盖）
    with open(file_path, "w", encoding="utf-8") as outfile:
        for data in modified_data:
            outfile.write(json.dumps(data, ensure_ascii=False) + "\n")
    
    print(f"\n完成！已将 {len(need_reclassify)} 条重新分类的数据写入 {file_path}")

In [47]:
file_path = "./datasets/train.jsonl"
model = "mimo-v2-flash"

reclassify(model, file_path)

发现 18 条数据需要重新分类
[1/18] 当前数据需要重新分类：
  原分类: 科幻小说
  内容: 研究员李志远受《盗梦空间》启发，试图通过改造国际象棋的“车”来验证梦境与现实的物理法则差异。他让助手小萝莉在梦中推倒重心偏移的棋子，结果棋子在现实中倒向了与梦境相反的方向，证明了她身处现实。

然而，小萝莉随即惊醒并指出，她在梦中预判了棋子会转回原位，这种对梦境逻辑的反向认知能力，反而证明了她当时处于清醒的梦境控制中。李志远由此意识到，小萝莉拥有罕见的梦境自控力。这次实验让他深刻体会到国际象棋与梦境探索中蕴含的深层智慧，坚定了他继续研究的决心。...
  → 新分类: 文学小说, 分类理由：文本通过塑造研究员李志远和助手小萝莉的人物形象，构建了关于梦境实验的故事情节，描绘了梦境与现实的具体环境，艺术地探讨了认知与现实的主题，符合文学小说通过人物、情节和环境反映社会生活的定义。
[2/18] 当前数据需要重新分类：
  原分类: 医学科普
  内容: 胡刚是大连医科大学附属第一医院的整形外科主任，医术精湛，尤其在脂肪移植和肉毒杆菌毒素应用方面有深入研究。一天，著名演员李燕因面部神经受损求助于他。胡刚采用创新手术，通过微创修复神经并精准注射肉毒毒素，成功让李燕恢复了自然笑容。

随后，胡刚又接手了消防员张伟的严重面部烧伤修复手术。术前，实验室的肉毒毒素因试剂问题活性降低，胡刚连夜排查并紧急重制，解决了危机。手术中，张伟突发血管痉挛，胡刚冷静应对，注射血管扩张剂并松解血管，成功化解险情。手术进行到关键时刻，手术室突遇停电，光线和氧气供应不足。胡刚带领团队利用应急灯和触觉继续手术，最终坚持到电力恢复，顺利完成手术。

术后，李燕前来致谢，与胡刚深入交谈。胡刚感慨，医学不仅是科学，更是责任与人文关怀。这次经历让他更加坚定了作为医者的信念：用精湛的医术和温暖的关怀，为患者带去希望。...
  → 新分类: 文学小说, 分类理由：文本通过塑造人物形象（胡刚医生）、构建故事情节（治疗患者、处理医疗危机）和描绘具体环境（医院、手术室），艺术地反映了医疗行业的社会生活和医者的人文关怀，符合文学小说的定义。
[3/18] 当前数据需要重新分类：
  原分类: 科幻小说
  内容: 海洋生物研究员陈旭痴迷于收集深海机械残骸。在一次探险中，他的团队在未探索海域发现了一块巨大的金属残骸，疑

In [48]:
count_classifications(file_path)

train.jsonl 总样本数: 3005

各分类数据统计:
  两性关系: 5 条 (0.2%)
  亲子育儿: 289 条 (9.6%)
  人文社科: 340 条 (11.3%)
  其他分类: 118 条 (3.9%)
  动漫幽默: 232 条 (7.7%)
  励志成长: 185 条 (6.2%)
  古籍地理: 48 条 (1.6%)
  外语学习: 6 条 (0.2%)
  家居生活: 17 条 (0.6%)
  家庭教育: 41 条 (1.4%)
  政治军事: 18 条 (0.6%)
  文学小说: 865 条 (28.8%)
  旅游休闲: 18 条 (0.6%)
  法律哲学: 15 条 (0.5%)
  生活时尚: 12 条 (0.4%)
  科普读物: 165 条 (5.5%)
  经济管理: 58 条 (1.9%)
  自然科学: 291 条 (9.7%)
  艺术收藏: 166 条 (5.5%)
  青春文学: 116 条 (3.9%)


In [55]:
synthesis_file_path = "../synthesis/persona_driven/output/persona_driven_sft.jsonl"
with open(synthesis_file_path, "r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line)
        category = data["category"]
        text = data["messages"][1]["content"]
        print(f"合成数据：{text}\n")
        classify_result = get_classify_response(client, model, CLASSIFY_SYSTEM_PROMPT, text)
        print(f"当前分类：{category}，分类结果：{classify_result}")
        print("=" * 30)

合成数据：- 分析《银魂》第1话吐槽逻辑：拆解坂田银时对日常危机的反讽回应。预期成果：学生掌握“夸张+反差”结构。成果指标：完成5个吐槽练习，自评准确率≥80%。
- 模仿《齐木楠雄的灾难》第1话超能力梗：将普通场景转为夸张误会。预期成果：学生独立创作1个幽默段子。成果指标：提交1个段子，幽默度≥7/10（自评）。
- 整合与优化：使用Notion整理梗库（每周1次），将上述案例归档并混搭创作。预期成果：形成可复用素材库。成果指标：库内≥10条梗，复用率≥30%。

结果验收标准：提交1份含3个原创动漫幽默短剧的PDF，总字数≤800字，幽默度≥7/10（自评+互评）。

当前分类：动漫幽默，分类结果：('动漫幽默', '理由：文本内容围绕动漫作品《银魂》和《齐木楠雄的灾难》的幽默逻辑进行分析与创作指导，涉及“吐槽”、“反讽”、“夸张误会”等幽默元素，旨在创作动漫相关的幽默短剧，符合“内容与动画、漫画相关，或以制造笑料、体现幽默感为主要目的”的定义。')
合成数据：AI伦理：风险警示与创新机遇的两种视角

作为一名曾在知名科研机构工作的科普研究者，我持续关注科学传播前沿。人工智能伦理对科普创作与传播的影响，可从两种视角审视。

风险警示视角：AI生成内容可能传播伪科学，算法推荐易制造信息茧房，导致公众误解复杂科学议题。版权与数据隐私问题削弱科普公信力，自动化写作或使内容同质化，丧失人文温度。科普工作者需警惕AI“幻觉”，避免未经核实的输出误导受众，尤其在健康与环境等敏感领域。

创新机遇视角：AI赋能个性化科普，通过自然语言处理与虚拟现实，实现互动式、沉浸式知识传播，提升公众参与度。智能工具辅助数据可视化与实验模拟，降低创作门槛，让前沿科学更易触达大众。伦理框架下，AI可优化多语言翻译，促进全球科学普及。

结论：在当前行业前沿，科普工作者应以风险警示为底线，建立AI内容审核机制；同时拥抱创新机遇，利用AI增强互动与包容性。平衡二者，方能推动负责任的科学传播，服务公众利益。

当前分类：科普读物，分类结果：('科普读物', '理由：文本围绕人工智能伦理对科普创作与传播的影响展开，从风险与机遇两个视角进行分析，并提出结论，内容核心是向非专业读者普及科学知识（AI伦理）及科学传播方法，符合“科普读物”中“以通俗易懂的方式向非专业读者普及科学知识、传播科学思想”的定义。'